# Serialization

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

import json
import math
import os

import pandas as pd
from IPython.display import Markdown, display
from tensorwaves.function.sympy import create_function
from tqdm.auto import tqdm

from polarization import formulate_polarization
from polarization.amplitude import DalitzPlotDecompositionBuilder
from polarization.data import create_data_transformer, generate_meshgrid_sample
from polarization.io import (
    export_polarization_field,
    mute_jax_warnings,
    perform_cached_doit,
)
from polarization.lhcb import _load_model_parameters, load_three_body_decays

mute_jax_warnings()

reference_subsystem = 1
dynamics_configurator = load_three_body_decays("../../data/isobars.json")
decay = dynamics_configurator.decay
amplitude_builder = DalitzPlotDecompositionBuilder(decay)
amplitude_builder.dynamics_choices = dynamics_configurator
model = amplitude_builder.formulate(reference_subsystem)
imported_parameter_values = _load_model_parameters(
    "../../data/modelparameters.json", decay
)
model.parameter_defaults.update(imported_parameter_values)

In [ ]:
polarization_exprs = formulate_polarization(amplitude_builder, reference_subsystem)
unfolded_polarization_exprs = [
    perform_cached_doit(expr.doit().xreplace(model.amplitudes))
    for expr in tqdm(polarization_exprs, desc="Unfolding polarization expressions")
]
unfolded_intensity_expr = perform_cached_doit(model.full_expression)

In [ ]:
polarization_funcs = [
    create_function(expr.xreplace(model.parameter_defaults), backend="jax")
    for expr in tqdm(unfolded_polarization_exprs)
]
intensity_func = create_function(
    unfolded_intensity_expr.xreplace(model.parameter_defaults),
    backend="jax",
)

In [ ]:
transformer = create_data_transformer(model)
grid_sample = generate_meshgrid_sample(decay, resolution=100)
grid_sample = transformer(grid_sample)
X = grid_sample["sigma1"]
Y = grid_sample["sigma2"]

In [ ]:
polarization_grids = [func(grid_sample).real for func in polarization_funcs]
alpha_x = polarization_grids[0]
df = pd.DataFrame(alpha_x, index=X[0], columns=Y[:, 0])
df.to_json("alpha-x-pandas.json")
df.to_json("alpha-x-pandas-json.zip", compression={"method": "zip"})
df.to_csv("alpha-x-pandas.csv")

df_dict = df.to_dict()
filtered_df_dict = {
    x: {y: v for y, v in row.items() if not math.isnan(v)} for x, row in df_dict.items()
}
with open("alpha-x-python.json", "w") as f:
    json.dump(filtered_df_dict, f)

json_dict = dict(
    x=X[0].tolist(),
    y=Y[:, 0].tolist(),
    z=alpha_x.tolist(),
)
with open("alpha-x-arrays.json", "w") as f:
    json.dump(json_dict, f, separators=(",", ":"))

In [ ]:
def render_kilobytes(path, markdown: bool = False) -> str:
    byt = os.path.getsize(path)
    kb = f"{1e-3*byt:.0f}"
    if markdown:
        return f"\n - **{kb} kB**: {{download}}`{path}`"
    return f"\n  {kb:>4s} kB  {path}"


src = f"File sizes for {len(X[0])}x{len(Y[:, 0])} grid:"
markdown = "EXECUTE_NB" in os.environ
src += render_kilobytes("alpha-x-arrays.json", markdown)
src += render_kilobytes("alpha-x-pandas.json", markdown)
src += render_kilobytes("alpha-x-python.json", markdown)
src += render_kilobytes("alpha-x-pandas-json.zip", markdown)
src += render_kilobytes("alpha-x-pandas.csv", markdown)
if markdown:
    display(Markdown(src))
else:
    print(src)

## Exported polarization grids

:::{note}
Decided to use the `alpha-x-arrays.json` format. It can be exported with {func}`.export_polarization_field`.
:::

In [ ]:
export_polarization_field(
    sigma1=X[0],
    sigma2=Y[:, 0],
    alpha_x=polarization_grids[0],
    alpha_y=polarization_grids[1],
    alpha_z=polarization_grids[2],
    intensity=intensity_func(grid_sample),
    filename="polarizations-model-0.json",
)

In [ ]:
if "EXECUTE_NB" in os.environ:
    src = (
        "Polarization grid can be downloaded here:"
        " {download}`polarizations-model-0.json`."
    )
    display(Markdown(src))